In [1]:
# Install pip packages in the current Jupyter kernel
# Uncomment if needed

# import sys
# !{sys.executable} -m pip install --quiet --upgrade pip 
# !{sys.executable} -m pip install --quiet tensorflow 
# !{sys.executable} -m pip install --quiet matplotlib

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

2023-09-11 20:04:13.334404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load the model from the disk

In [3]:
MODEL_VERSION = 7
model = tf.keras.models.load_model(f'./saved-models/models/{MODEL_VERSION}')

2023-09-11 20:04:18.352128: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-11 20:04:18.409976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-11 20:04:18.410475: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-11 20:04:18.414518: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-11 20:04:18.415100: I tensorflow/compile

# Constants and data

In [4]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3  # The RGB channels
EPOCHS = 50

In [5]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "data",
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE    
)
dataset

Found 2152 files belonging to 3 classes.


<_BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [6]:
len(list(dataset))

2023-09-11 20:04:33.855512: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25165824 exceeds 10% of free system memory.
2023-09-11 20:04:33.904828: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25165824 exceeds 10% of free system memory.
2023-09-11 20:04:33.957516: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25165824 exceeds 10% of free system memory.
2023-09-11 20:04:34.003785: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25165824 exceeds 10% of free system memory.
2023-09-11 20:04:34.060991: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25165824 exceeds 10% of free system memory.


68

In [7]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

## Let's find the wrong predictions

In [8]:
wrong_predictions = []

# We check in the whole dataset
j = 1
print("Batches:")
for images, labels in dataset.take(-1):
    # Predict a batch
    predictions = model.predict(images.numpy(), verbose=0)
    print(j, end=".. ") 
    j += 1

    for label, prediction, image in zip(labels, predictions, images):
        predicted_label = np.argmax(prediction)
        if predicted_label != label:
            wrong_predictions.append((image, class_names[label], class_names[predicted_label], round(100 * prediction[predicted_label], 2))) 

num_wrongs = len(wrong_predictions)
lines = int(np.ceil(num_wrongs / 3))
print("\n\n", num_wrongs, "wrong predictions out of", dataset.cardinality().numpy(), "images")

Batches:


2023-09-11 20:04:53.758386: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


1.. 2.. 3.. 4.. 5.. 6.. 7.. 8.. 9.. 10.. 11.. 12.. 13.. 14.. 15.. 16.. 17.. 18.. 19.. 20.. 21.. 22.. 23.. 24.. 25.. 26.. 27.. 28.. 29.. 30.. 31.. 32.. 33.. 34.. 35.. 36.. 37.. 38.. 39.. 40.. 41.. 42.. 43.. 44.. 45.. 46.. 47.. 48.. 49.. 50.. 51.. 52.. 53.. 54.. 55.. 56.. 57.. 58.. 59.. 60.. 61.. 62.. 63.. 64.. 65.. 66.. 67.. 68.. 

 13 wrong predictions out of 68 images


In [9]:
# Comment only to save space on GitHub
# Uncomment if needed

# plt.figure(figsize=(15, 5 * lines))
# for i in range(num_wrongs):
#     ax = plt.subplot(lines, 3, i + 1)

#     image, actual_class, predicted_class, confidence = wrong_predictions[i]
#     plt.imshow(image.numpy().astype("uint8"))
    
#     plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
    
#     plt.axis("off")